In [ ]:
!/usr/local/cuda/bin/nvcc --version
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc_plugin
!cuda-install-samples-11.2.sh ~ && cd /root/NVIDIA_CUDA-11.2_Samples

In [ ]:
%%cuda --name curand.cu
#include <omp.h>
#include <stdio.h>
#include <iostream>
#include <iomanip>
#include <time.h>

void matrixmult( int **m1, int m1_row, int m1_col, int **m2, int m2_row, int m2_col, int **m3, int m3_row, int m3_col);
 
void matrixfill( int **m, int row, int col)
{
    for (int i = 0; i < row; ++i)
        for (int j = 0; j < col; ++j)
        {
            m[i][j]=1;
        }
}

void print_matr(int **m, int row, int col)
{
    for (int i = 0; i < row; ++i)
    {
        for (int j = 0; j < col; ++j)
            std::cout << std::setw(3) << m[i][j] << ' ';
        std::cout << std::endl;
    }
}
 
int main()
{
	int row1 = 128;
  int col1 = 128;
  int row2 = 0;
  int col2 = 0;
  int row3 = row1;
  int col3 = col2;
 
	clock_t start, stop;
 
  int **arr1 = new int*[row1];
  int **arr2 = new int*[row2];
  int **arr3 = new int*[row3];
 
	for (int i = 0; i < row1; i++)
            arr1[i] = new int[col1];
 
    matrixfill(arr1, row1, col1);
 
    
    for (int i = 0; i < row2; i++)
            arr2[i] = new int[col2];
 
    matrixfill(arr2, row2, col2);
    
    for (int i = 0; i < row3; i++)
            arr3[i] = new int[col3];
 
    for (int i = 0; i < row3; ++i)
      for (int j = 0; j < col3; ++j)
        arr3[i][j] = 0;
   
	start = clock();

	//если число столбцов первой матрицы равно числу строк второй матрицы
    if(col1 == row2)
        matrixmult(arr1, row1, col1, arr2, row2, col2, arr3, row3, col3);
    else{
        std::cerr << " Error! " << std::endl;
        return 1;}
 
    std::cout << "\nResult matrix:" << std::endl;
    print_matr(arr3, row3, col3);


    for(int i = 0; i < row1; i++)
        delete[] arr1[i];
    delete[] arr1;
    for(int i = 0; i < row2; i++)
        delete[] arr2[i];
    delete[] arr2;
    for(int i = 0; i < row3 ; i++)
        delete[] arr3[i];
    delete[] arr3;
 
	stop = clock();
	printf("Time=%f sec.\n",((double)(stop-start)/1000.0));
    return 0;
}
 
 
void gpu_blas_mmul(const float* A, const float* B, float* C, const int m, const int k, const int n) {
int stage;
#pragma omp parallel private(stage) shared(A, B, C)
{
        int n_threads = omp_get_num_threads();
        *save = n_threads;  // used for printing in main()
        int n_blocks = sqrt(n_threads);
        int block_size = m_size / n_blocks
        int PrNum = omp_get_thread_num();
        int i1 = PrNum / n_blocks, j1 = PrNum % n_blocks;
        double *A1, *B1, *C1;
        for (stage = 0; stage < n_blocks; ++stage) {
            A1 = A + (i1 * m_size + ((i1 + stage) % n_blocks)) * block_size;
            B1 = B + (((i1 + stage) % n_blocks) * m_size + j1) * block_size;
            C1 = C + (i1 * m_size + j1) * block_size;
            for (int i = 0; i < block_size; ++i) {
                for (int j = 0; j < block_size; ++j) {
                    for (int k = 0; k < block_size; ++k) {
                        *(C1 + i * m_size + j) += *(A1 + i * m_size + k) * *(B1 + k * m_size + j);
                    }
                }   
            }
        }
    }
void matrixmult( int **m1, int m1_row, int m1_col, int **m2, int m2_row, int m2_col, int **m3, int m3_row, int m3_col)
{

	omp_set_num_threads(12);	
	int temp = 0;
	int i=0, j=0, k=0;

	#pragma omp parallel 
	{	
		int num = omp_get_thread_num();
		#pragma omp parallel for schedule(static)  private(k) 
		for (k = 0;k < m2_col; ++k)
			#pragma omp parallel for schedule(static)  private(i)
			for (i = 0;i < m1_row; ++i)
			#pragma omp parallel for schedule(static) shared(m1, m2, m3) private(j) reduction(+:temp)	
				for (j = 0;j < m1_col; ++j)
				{
					temp += m1[i][j] * m2[j][k];
					m3[i][k] = temp;
				}					
	}
						
}

In [ ]:
!nvcc -o /content/src/consistently /content/src/consistently.cu -lcurand -lcublas

In [ ]:
!/content/src/consistently